In [253]:
import numpy as np
import pandas as pd

## Importing data
This reads the spreadsheet stored at `data/root.xlsx`. The first step is to replace the column names to make it more amenable to programmatic referencing.

In [254]:
PATH = "data/root.xlsx"
SHEET = "Data"

# read
df = pd.read_excel(PATH, SHEET)
df.columns = [
    "nhs_num",
    "lab_num",
    "age",
    "gender",
    "location",
    "location_other",
    "size",
    "mdm2",
    "karyo_code",
    "diagnosis",
    "diagnosis_other",
    "follow_up",
    "mortality",
    "cancer_mortality",
    "comment",
    "query",
    "misc"
]

df

/opt/miniconda3/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,nhs_num,lab_num,age,gender,location,location_other,size,mdm2,karyo_code,diagnosis,diagnosis_other,follow_up,mortality,cancer_mortality,comment,query,misc
0,7079315611,LH24-19172,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 normal,NaN,NaN,NaN,alive,NaN,CF,NaN,NaN
1,4509208588,LH24-20991,74,Female,O - Other (Free- text),Breast,20,Non-amplified,MDM2 normal,Lipoma,NaN,NaN,alive,NaN,CF,NaN,NaN
2,6342778353,LH24-18651,45,Female,HN - Head and Neck,Right cheek parotid gland,71,Non-amplified,MDM2 normal,Lipoma,NaN,NaN,alive,NaN,CF,Should parotid gland be in other?,NaN
3,6415674862,LH24-20355,23,Female,LLS - Lower limbs (Superficial),NaN,punch biopsy,Non-amplified,MDM2 normal,Other (free text),Myxoid liposarcoma,NaN,alive,NaN,CF,Unsure whether diagnosis fits in option 'Pleom...,NaN
4,4520257115,LH24-19486,73,Female,ULS - Upper limbs (Superficial),NaN,15,Non-amplified,MDM2 normal,Lipoma,NaN,NaN,alive,NaN,CF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,6049456305,BH09-3046,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865,4064416066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866,4343743969,LH09-10296,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,4382331137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning data
Defining some common column cleaning utilities. Excel stores some blank cells with `\xa0`. These must be converted into plain empty strings - something that will be repeated for other fields.

In [255]:
def std_clean(df, col, is_str=True):
    if is_str:
        df[col] = df[col].str.strip().str.lower()
    df.loc[df[col] == "\xa0", col] = np.nan
    df.loc[df[col] == "", col] = np.nan

### Age
*TODO - duplication*

In [256]:
std_clean(df, "age", is_str=False)

df["age"].unique()

array([nan, 74, 45, 23, 73, 79, 68, 70, 57, 38, 56, 28, 75, 41, 82, 69,
       52, 12, 85, 55, 65, 64, 89, 42, 35, 80, 54, 43, 63, 51, 16, 61, 46,
       91, 86, 47, 33, 67, 62, 66, 77, 76, 39, 44, 81, 84, 60, 50, 83, 87,
       48, 32, 19, 26, 31, 49, 40, 53, 58, 22, 59, 34, 72, 25, 71, 15, 27,
       18, 29, 78, 36, 37, 88, 13, 98, 100, 17, 30, 20, 93, 90, 92, 94, 6,
       1, 4, 'duplication of above', 'duplication', 8, 24, 21, 14],
      dtype=object)

### Gender
Standardising gender into "m", "f" and NaN for missing or "unspecified" data.

In [257]:
std_clean(df, "gender")
df.loc[df["gender"] == "unspecified", "gender"] = np.nan
df.loc[df["gender"] == "male", "gender"] = "m"
df.loc[df["gender"] == "female", "gender"] = "f"

df["gender"].unique()

array([nan, 'f', 'm'], dtype=object)

### Location
The spreadsheet encodes this in the format `{code} - {description}`. Since the description is the same for every code, we are filtering only the code.

In [258]:
std_clean(df, "location")
# NaN modification
idxs = ~df["location"].isna()
df.loc[idxs, "location"] = df.loc[idxs, "location"] \
    .apply(lambda x: x.split("-")[0].strip())

df["location"].unique()

array([nan, 'o', 'hn', 'lls', 'uls', 'uld', 't', 'lld'], dtype=object)

### Location (other)
*TODO*

In [259]:
std_clean(df, "location_other")

df["location_other"].unique()

array([nan, 'breast', 'right cheek parotid gland', 'brerat',
       'spermatic cord', 'left testicle', 'right kidney', 'groin',
       'right parotid', 'left hemiscrotum', 'left groin', 'right groin',
       'tongue', 'thrombosed varix', 'left pararenal mass',
       'shoulder blade soft tissue', 'left kidney', 'left paraspinal',
       'right 6th rib', 'presacral', 'r posterior shoulder',
       'left adrenal gland', 'right posterior arm, shoulder plus back',
       'axilla', 'retroperitoneal', 'left sided thymic fat', 'thymus',
       'inguinal fat (left scrotal hernia)', 'vaginal wall polyp',
       'axillary core biopsy', 'small bowel mesenteric mass',
       'mesenteric mass', 'hilar mass', 'anterior mediastinal mass',
       'plueral debris', 'left scrotal lesion', 'inguinal orchidectomy',
       'right retroperitoneum (iliopsoas)', 'breast l',
       'right inguinal orchidectomy', 'mediastinal mass',
       'retroperitoneum',
       'right inguinal orchidectomy plus scrotal lesi

### Size
*TODO - split*

In [260]:
std_clean(df, "size")

df["size"].unique()

array([nan, 'punch biopsy', 'excisional biopsy - no macro',
       'excision two pieces: 116 and 200', 'shave excision  - 6',
       'core biopsies - 11', 'core biopsy - 13', 'excision - 220',
       'core biopsy', 'core biopsy - 15', 'excision - 205',
       'core biopsy - 17', 'core biospy - 14', 'excision - 95',
       'omental biopsy', 'excision - 90', 'core biopsy - 20',
       'core biospy - 12', 'excision - 10', 'excision - 60',
       'excision - 120', 'core biospy - 10', 'excisional biopsy - 14',
       'core biospy - no macro', 'excision - 25', 'excision - 80',
       'core biopsy - 22', 'core biopsy - 10', 'excision - 125',
       'core biospy - 17', 'excision - 115', 'core biopsy - 12',
       'excision - 140', 'core biopsy - 14', 'core biopsy - 16',
       'core biopsy - 5', 'biopsy - 8',
       'excision - no macro frozen section - 90', 'core biopsy - 221',
       'biopsy - 15', 'excision biopsy - 50', 'excision - 17',
       'excision - 170', 'blunt dissection/excision -

### MDM2
Now transformed to represent MDM2 amplification as a boolean.

In [261]:
std_clean(df, "mdm2")
df.loc[df["mdm2"] == "non-amplified", "mdm2"] = False
df.loc[df["mdm2"] == "amplified", "mdm2"] = True

df["mdm2"].unique()

array([nan, False, True], dtype=object)

### Karyo Code
*TODO - purpose?*

In [262]:
std_clean(df, "karyo_code")


df["karyo_code"].unique()

array(['mdm2 normal', 'mdm2 amp', 'fish failed', '13q normal',
       'equivocal', 'amplified', 'fish ab', 'fish normal', nan,
       'alk not rearranged', 'abnormal', 'dleu deleted'], dtype=object)

### Diagnosis
*TODO*

In [263]:
std_clean(df, "diagnosis")

df["diagnosis"].unique()

array([nan, 'lipoma', 'other (free text)',
       'well differentiated liposarcoma', 'liposarcoma',
       'atypical lipomatous tumour', 'dedifferentiated liposarcoma',
       'pleomorphic liposarcoma', 'sclerosing mesenteritis',
       'pleomorphic myxoid liposarcoma', 'low-grade leiomyosarcoma',
       'fibrolipoma', 'poorly differentiated sarcoma',
       'undifferentiated liposarcoma'], dtype=object)

### Diagnosis (Other)
*TODO*

In [264]:
std_clean(df, "diagnosis_other")

df["diagnosis_other"].unique()

array([nan, 'myxoid liposarcoma',
       'fibromycoid tumour of uncertain \nmalignant potential',
       'metastatic nonseminomatous germ cell \ntumour, teratoma with focal somatic malignancy (sarcoma, showing areas of rhabdomyloblastic differentiation)',
       'recurrent', 'lipoblastoma', 'scarring process',
       'atypical spindle cell lipomatous tumour', 'myolipoma',
       'igg4 related disease?', 'fibrous dysplasia',
       'atypical lipomatous tumour/well \ndifferentiated liposarcoma, grade 1',
       'atypical lipomatous tumour/well \ndifferentiated liposarcoma, grade 2',
       'fat necrosis',
       'borderline fibroblastic tumour with\nsome features concerning for malignancy',
       'spindle cell lipoma', 'solitary fibrous tumour',
       'spindle cell lipoma, with a minimal \nadipocytic component',
       'spindle cell soft tissue tumour of uncertain \nmalignant potential',
       'spindle cell', 'likely metastatic carcinoma',
       'localised type tenosynovial giant cel

### Follow Up
*TODO*

In [265]:
std_clean(df, "follow_up")

df["follow_up"].unique()

array([nan, 'death 2025.02.11', 'death 2024.02.03', 'death 2024.07.15',
       'death 2025.01.11', 'death 2024.03.29',
       'tumour right thigh mixed type im angioma',
       'tumour right arm myxofibrosarcoma',
       'pleomorphic atypical lipomatous tumour',
       'dedifferentiated liposarcoma fnclcc grade 3',
       'a) retroperitoneal resection: well-differentiated liposarcoma b) colon: within normal limits',
       'myxoid tumour with no features of malignancy',
       'recurrent liposarcoma posterior left thigh',
       'cellular angiofibroma', 'death 2023.12.30',
       'lipoma left shoulder', 'cellular agiofibroma',
       'spindle cell lipoma', 'right anterior shoulder haemangioma',
       'excision, mass back: grade 3 spindle cell sarcoma',
       'well differentiated liposarcoma with focal low-grade dedifferentiation',
       'high grade myxoid sarcoma', 'intramuscular lipomatous tumour',
       'well differentiated liposarcoma (fnclcc grade 1)',
       'undifferentiated 

### Mortality
*TODO - what to do about living and nans*

In [266]:
std_clean(df, "mortality")
idxs = ~df["mortality"].isna()
df.loc[idxs, "mortality"] = df.loc[idxs, "mortality"] \
    .apply(lambda x: -1 if x == "alive" else int(x.split()[1]))

df["mortality"].unique()

array([-1, 39, 56, 85, 83, 68, nan, 76, 82, 52, 61, 72, 77, 44, 87, 79,
       50, 101, 62, 55, 92, 58, 63, 65, 80, 35, 78, 40, 64, 93, 70, 74,
       69, 86, 34, 81, 96, 66, 57, 84, 89, 54, 59, 75, 51, 94, 67, 71, 73,
       90, 95, 17, 47, 88], dtype=object)

### Cancer Mortality
*TODO - split into yes/no and reason*

In [267]:
std_clean(df, "cancer_mortality")

df["cancer_mortality"].unique()

array([nan, 'no', 'yes, carcinoma',
       'yes, dedifferentiated liposarcoma of scrotum',
       'yes, necrotic undifferentiated malignancy',
       'yes, stage 4 high grade serous peritoneal cancer',
       'yes, de-differentiated liposarcoma of the \nperitoneum',
       'yes, grade 3 metastatic spindle cell sarcoma, mets to lung, liver, bone and kidney',
       'yes, myxoid sarcoma',
       'yes, metastatic retroperitoneal de-differentiated liposarcoma',
       'yes, lung mets from sarcoma of left thigh',
       'yes, poorly differntiated squamous cell carcinoma wiht a spindle cell component with brain mets',
       'yes recurrence spindle cell sarcoma of retroperitoneum and peritoneum',
       'yes pulmonary intimal sarcoma',
       'yes  stage 4b, carcinoma of the ovary (hepatic/pulmonary)',
       'yes high grade \nsarcoma in his right axilla',
       'yes . left sided epithelioid malignant mesothelioma -',
       'yes recurrent spindle cell sarcoma grade 2', 'yes osteosarcoma',


In [268]:
# TODO
# comment
# query
# misc

## Viewing the cleaned data

In [269]:
df.to_csv("data/clean.csv", index=False)
df

,nhs_num,lab_num,age,gender,location,location_other,size,mdm2,karyo_code,diagnosis,diagnosis_other,follow_up,mortality,cancer_mortality,comment,query,misc
0,7079315611,LH24-19172,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,-1,NaN,CF,NaN,NaN
1,4509208588,LH24-20991,74,f,o,breast,NaN,False,mdm2 normal,lipoma,NaN,NaN,-1,NaN,CF,NaN,NaN
2,6342778353,LH24-18651,45,f,hn,right cheek parotid gland,NaN,False,mdm2 normal,lipoma,NaN,NaN,-1,NaN,CF,Should parotid gland be in other?,NaN
3,6415674862,LH24-20355,23,f,lls,NaN,punch biopsy,False,mdm2 normal,other (free text),myxoid liposarcoma,NaN,-1,NaN,CF,Unsure whether diagnosis fits in option 'Pleom...,NaN
4,4520257115,LH24-19486,73,f,uls,NaN,NaN,False,mdm2 normal,lipoma,NaN,NaN,-1,NaN,CF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,6049456305,BH09-3046,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865,4064416066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866,4343743969,LH09-10296,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,4382331137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
